## importação dos pacotes


In [1]:
import pyodbc as PDB  # Drive para conexão com banco de dados
import pandas  as pd
import sys
import traceback


## variaveis de controle

In [2]:
server = '?'
database = '?'
username = '?'
password = '?'
driver= '{ODBC Driver 17 for SQL Server}'

diretorio_planilha = r'D:\1.Implanta(Nunca Apagar)\Power BI Implanta\Compressão Banco\Alterações Datetime2 para Date\TodasColunasDatetime2.xlsx'
colunas_datetime2 = pd.DataFrame()


## Utils


In [3]:
def get_conection(driver,server,database,username,password):
    _conexao = PDB.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    return _conexao



    

In [4]:
def executa_procedure(database_name,id,schemaName,tableName,column,dataType):
    dados = pd.DataFrame()
    try:
        _con = get_conection(driver,server,database_name,username,password)
        
        query  = "EXEC HealthCheck.uspCalculaAlteracaoCampoDatetime2ToDate @IdRegistro ={0},@SchemaName ='{1}',@TableName ='{2}',@Column ='{3}',@DataTypeAtual ='{4}'".format(id,schemaName,tableName,column,dataType)

        SQL_exec_procedure = pd.read_sql_query(query,_con)

        dados = pd.DataFrame(SQL_exec_procedure, columns=['Id','SchemaName','TableName','Coluna','Type','Indexable','QuantidadeLinhas','PossoAlterar', 
         'DatabasesQuePossueValorHora','DatabasesQueNaoPossueValorHora','DatabasesTableVazio'])
         
         
        dados.set_index('Id',inplace=True)
        _con.close() 
        return dados

    except TypeError:
        _con.close()
    return dados

In [5]:
def carregar_excel_tipo_dados():
    global colunas_datetime2
    colunas_datetime2 = pd.read_excel(diretorio_planilha,index_col=0)

    colunas_datetime2["SchemaName"]= colunas_datetime2["SchemaName"].astype(str) 
    colunas_datetime2["TableName"]= colunas_datetime2["TableName"].astype(str) 
    colunas_datetime2["Coluna"]= colunas_datetime2["Coluna"].astype(str) 
    colunas_datetime2["Type"]= colunas_datetime2["Type"].astype(str) 
    colunas_datetime2["Indexable"]= colunas_datetime2["Indexable"].astype(bool) 
    colunas_datetime2["QuantidadeLinhas"]= colunas_datetime2["QuantidadeLinhas"].astype(int) 
    colunas_datetime2["ExisteHoraMinutosSegundos"]= colunas_datetime2["ExisteHoraMinutosSegundos"].astype(bool) 
    colunas_datetime2["JaValidado"]= colunas_datetime2["JaValidado"].astype(bool)
    colunas_datetime2["QuantidadeBancos"]= colunas_datetime2["QuantidadeBancos"].astype(int)

    colunas_datetime2["DatabasesQuePossueValorHora"]= colunas_datetime2["DatabasesQuePossueValorHora"].astype(str) 
    colunas_datetime2["DatabasesQueNaoPossueValorHora"]= colunas_datetime2["DatabasesQueNaoPossueValorHora"].astype(str) 
    colunas_datetime2["DatabasesTableVazio"]= colunas_datetime2["DatabasesTableVazio"].astype(str) 
    
    colunas_datetime2["DatabasesQuePossueValorHora"].fillna("", inplace = True) 
    colunas_datetime2["DatabasesQueNaoPossueValorHora"].fillna("", inplace = True) 
    colunas_datetime2["DatabasesTableVazio"].fillna("", inplace = True) 

    




## Executa apenas para testar

In [6]:
dados =  executa_procedure('db_name',1,'Processo','ProcessosAndamentos','DataPrazo','datetime2')

In [7]:
dados

,SchemaName,TableName,Coluna,Type,Indexable,QuantidadeLinhas,PossoAlterar,DatabasesQuePossueValorHora,DatabasesQueNaoPossueValorHora,DatabasesTableVazio
Id,,,,,,,,,,
1,Processo,ProcessosAndamentos,DataPrazo,datetime2,True,175420,False,apresentacao-siscaf.implanta.net.br,,


## Conecta no servidor 

In [34]:
database = 'master'

conexao =get_conection(driver,server,database,username,password)
 

## Testa a Conexão

In [41]:
cursor = conexao.cursor()


cursor.execute("SELECT @@Version") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

cursor.close()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Oct  1 2020 18:48:35 
	Copyright (C) 2019 Microsoft Corporation



## Monta a query para ler todos so bancos de dados

In [10]:
SQL_Query = pd.read_sql_query(
'''SELECT  D.database_id,D.name FROM  sys.databases AS D
 order by D.database_id''', conexao)

##Monta o DataFrame

In [11]:
databases = pd.DataFrame(SQL_Query, columns=['database_id','name'])

In [12]:
databases.count()

database_id    511
name           511
dtype: int64

## Executa um filtro para executar as querys somente nos databases selecionados

In [13]:
bancos_filtrados = databases[~databases['name'].str.contains("master|Manager|ajuda|Conversor|conversor|Copy|DNE|automationjobs|Configuracao|rglab|ESPELHO|apresentacao|COPY|conrerp|conter-br|core-ap|cra-mg|cra-rj|crb-01|creci-es|creci-rj|crb|cfb|hml|crtr|crq-go|crq-ba|cau-al|crefito-pi|crp-mg|cref-rs|2020|cress-ap")]

In [14]:
bancos_filtrados.count()

database_id    387
name           387
dtype: int64

In [17]:
bancos_filtrados.sample(20)

,database_id,name
209,213,cra-pr.implanta.net.br
244,248,crm-es.implanta.net.br
98,102,coren-df.implanta.net.br
228,232,crf-am.implanta.net.br
405,409,cro-to.implanta.net.br
415,419,crf-ms.implanta.net.br
368,372,crq-ms.implanta.net.br
113,117,cress-ms.implanta.net.br
273,277,crq-am.implanta.net.br
112,116,cau-rr.implanta.net.br


##Executa um for each para cada coluna datetime2

In [94]:
try:
    carregar_excel_tipo_dados()
    clone = colunas_datetime2

    for index, row in clone[1294:].iterrows():
        idregistro  =index
        quatidadeLinhas  =row['QuantidadeLinhas']
        ExisteHoraMinutosSegundos = row['ExisteHoraMinutosSegundos']
        JaValidado = row['JaValidado']
        
        if(ExisteHoraMinutosSegundos ==False):
            databasesQuePossueValorHora =row['DatabasesQuePossueValorHora']
            databasesQueNaoPossueValorHora =row['DatabasesQueNaoPossueValorHora']
            databasesTableVazio =row['DatabasesTableVazio'],
            schemaName =row['SchemaName']
            tableName =row['TableName']
            coluna =row['Coluna']
            print("==========================================>")
            print("===>Schema:",schemaName,"Table:",tableName,"Coluna:",coluna,"idregistro",idregistro)

            for i, linha in bancos_filtrados.sort_values("name").iterrows():
                
                if(clone.at[idregistro,'ExisteHoraMinutosSegundos'] ==True):
                    continue
                
                
                retorno = executa_procedure(linha['name'],idregistro,schemaName,tableName,coluna,row['Type'])
                
                clone.at[idregistro,'QuantidadeBancos'] += 1
                clone.at[idregistro,'QuantidadeLinhas'] =  quatidadeLinhas + int(retorno.loc[idregistro].QuantidadeLinhas)
                possoAlterar = bool(retorno.loc[idregistro].PossoAlterar)
                if(possoAlterar ==False):
                    clone.at[idregistro,'ExisteHoraMinutosSegundos'] = True
                    clone.at[idregistro,'JaValidado'] = True
                    clone.at[idregistro,'DatabasesQuePossueValorHora']  += "|"+linha['name']
                
                else:
                    if(int(retorno.loc[idregistro].QuantidadeLinhas) ==0):
                        clone.at[idregistro,'DatabasesTableVazio']  += "|"+linha['name']
                    else:
                        clone.at[idregistro,'DatabasesQueNaoPossueValorHora']  += "|"+linha['name']

                # print(linha['name'],"Existe HH:",clone.at[idregistro,'ExisteHoraMinutosSegundos'],'Vazio:' "Sim" if  clone.at[idregistro,'QuantidadeLinhas']==0 else 'Não' ) 
        
        clone.at[idregistro,'JaValidado'] = True   

                

except BaseException as ex:
    # Get current system exception
    ex_type, ex_value, ex_traceback = sys.exc_info()

    # Extract unformatter stack traces as tuples
    trace_back = traceback.extract_tb(ex_traceback)

    # Format stacktrace
    stack_trace = list()

    for trace in trace_back:
        stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))

    print("Exception type : %s " % ex_type.__name__)
    print("Exception message : %s" %ex_value)
    print("Stack trace : %s" %stack_trace)
    print(linha['name'],tableName,coluna,idregistro)         
            
            
        





==========================================>
===>Schema: TCU Table: PlanejamentosResultados Coluna: DataCadastro idregistro 1297
==========================================>
===>Schema: Compra Table: OrdensServicos Coluna: Data idregistro 1298
==========================================>
===>Schema: Compra Table: OrdensServicos Coluna: DataPrevisao idregistro 1299
==========================================>
===>Schema: Compra Table: OrdensServicos Coluna: DataVencimentoNotaFiscal idregistro 1300
==========================================>
===>Schema: Compra Table: OrdensServicos Coluna: DataEnvioEmail idregistro 1301
==========================================>
===>Schema: Patrimonio Table: BensMoveisReparos Coluna: Data idregistro 1302
==========================================>
===>Schema: Patrimonio Table: BensMoveisReparos Coluna: DataInicio idregistro 1303
==========================================>
===>Schema: Patrimonio Table: BensMoveisReparos Coluna: DataTermino idregistro 1304
==

##teste erro


In [95]:
def get_validation_query(row):
    return "SELECT L.{0},CAST(L.{0} AS TIME) FROM {1}.{2} AS L WHERE CAST(L.{0} AS TIME) <> '00:00:00.0000000'".format(row['Coluna'],row['SchemaName'],row['TableName'])

In [96]:
# Use .apply to save the new column if we'd like
clone['QueryValidacao'] = clone.apply(get_validation_query, axis=1)


In [97]:
clone.query("QuantidadeBancos > 0 and ExisteHoraMinutosSegundos == False and JaValidado ==True")

,SchemaName,TableName,Coluna,Type,Indexable,QuantidadeLinhas,JaValidado,QuantidadeBancos,ExisteHoraMinutosSegundos,DatabasesQuePossueValorHora,DatabasesQueNaoPossueValorHora,DatabasesTableVazio,QueryValidacao
id,,,,,,,,,,,,,
1297,TCU,PlanejamentosResultados,DataCadastro,datetime2,True,3,True,387,False,nan,nan|cau-ac.implanta.net.br|cau-am.implanta.net...,nan|caa-ma.implanta.net.br|cfa-br.implanta.net...,"SELECT L.DataCadastro,CAST(L.DataCadastro AS T..."
1299,Compra,OrdensServicos,DataPrevisao,datetime2,True,252,True,387,False,nan,nan|cau-br.implanta.net.br|cau-mg.implanta.net...,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.DataPrevisao,CAST(L.DataPrevisao AS T..."
1304,Patrimonio,BensMoveisReparos,DataTermino,datetime2,True,0,True,387,False,nan,nan|cau-go.implanta.net.br|cau-sc.implanta.net...,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.DataTermino,CAST(L.DataTermino AS TIM..."
1305,Financeiro,ConfiguracoesGeracoesDebitos,DataInicioVigencia,datetime2,True,12,True,387,False,nan,nan|cra-al.implanta.net.br|cra-am.implanta.net...,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.DataInicioVigencia,CAST(L.DataInicioV..."
1306,Financeiro,ConfiguracoesGeracoesDebitos,DataTerminoVigencia,datetime2,True,12,True,387,False,nan,nan|cra-al.implanta.net.br|cra-am.implanta.net...,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.DataTerminoVigencia,CAST(L.DataTermin..."
1310,Financeiro,ArquivosConciliacoesCieloHistoricosErros,ArquivoDataPrevistaPagamento,datetime2,True,672,True,387,False,nan,nan|cra-am.implanta.net.br|cra-df.implanta.net...,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.ArquivoDataPrevistaPagamento,CAST(L.A..."
1311,Financeiro,ArquivosConciliacoesCieloHistoricosErros,ArquivoDataVenda,datetime2,True,672,True,387,False,nan,nan|cra-am.implanta.net.br|cra-df.implanta.net...,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.ArquivoDataVenda,CAST(L.ArquivoDataVe..."
1317,Projeto,HistoricosSituacoes,DataCriacao,datetime2,True,0,True,387,False,nan,nan,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.DataCriacao,CAST(L.DataCriacao AS TIM..."
1322,Patrimonio,BensMoveisSeguros,DataFinalVigencia,datetime2,True,0,True,387,False,nan,nan|cau-go.implanta.net.br|cau-sp.implanta.net...,nan|caa-ma.implanta.net.br|cau-ac.implanta.net...,"SELECT L.DataFinalVigencia,CAST(L.DataFinalVig..."


## gera a planilha com o primeiro resultado

In [98]:
clone.query("QuantidadeBancos > 0 and ExisteHoraMinutosSegundos == False and JaValidado ==True").to_excel(r'D:\1.Implanta(Nunca Apagar)\Power BI Implanta\Compressão Banco\Alterações Datetime2 para Date\CalculoCamposDate_V2_4.xlsx', index = True)

##Fecha a conexão 

## Retorna somente os campos que podem ser alterados

In [ ]:
conexao.close()